## Mining argumentative content from Wikipedia

This notebook demonstrates how to use Project Debater APIs to extract the most relevant argumentative statements for a given topic from Wikipedia. 

The output includes three main parts:
1. A table of the most argumentative statements with respect to the topic
2. A narrative which automatically generated from the most argumentative statements
3. Given an additional list of aspects of interest it provides a summary of the most argumentative statements for each aspect seperately.


The following two lines define the topic of interest and its specific aspects that are of interest.  In this case, we are interested in pros and cons related to Roman Empire in general, and also those that related to war, economy and technology aspects of the Roman Empire.

In [1]:
dc = 'Roman Empire'
aspects = ['War','Economy','Technology']


These are additional parameters, which can remain as default in most cases:

In [2]:
articles_filename = dc + "_wikipedia.json"
dc_re = dc  # This can allow a more refined regular expression to find relevant sentences containing the topic concept.
            # For example 'Hybrid.*Cloud' would increase coverage.
topic = 'The ' + dc + ' was beneficial to the world'
max_sentence_length = 50
max_articles = 10000


### Install the early access program SDK

In [3]:
print('Install Early-Access-Program SDK:')
!wget -P . https://early-access-program.debater.res.ibm.com/sdk/python_api.tar.gz
!tar -xvf python_api.tar.gz
!cd python_api ; pip install .
!rm -f python_api.tar.gz*


Install Early-Access-Program SDK:
--2021-07-25 16:05:02--  https://early-access-program.debater.res.ibm.com/sdk/python_api.tar.gz
Resolving early-access-program.debater.res.ibm.com (early-access-program.debater.res.ibm.com)... 169.55.93.82
Connecting to early-access-program.debater.res.ibm.com (early-access-program.debater.res.ibm.com)|169.55.93.82|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 441774 (431K) [application/x-gzip]
Saving to: ‘./python_api.tar.gz’

python_api.tar.gz   100%[===================>] 431.42K   553KB/s    in 0.8s    

2021-07-25 16:05:04 (553 KB/s) - ‘./python_api.tar.gz’ saved [441774/441774]

x python_api/
x python_api/__init__.py
x python_api/setup.py
x python_api/README.md
x python_api/debater_python_api/
x python_api/debater_python_api/__init__.py
x python_api/debater_python_api/examples/
x python_api/debater_python_api/examples/__init__.py
x python_api/debater_python_api/examples/evidence_detection_example.py
x python_api/debater_p

  Created wheel for debater-python-api: filename=debater_python_api-3.5.8.RELEASE-py3-none-any.whl size=87347 sha256=610f85b95c2805f573f09db8e1c57720afcf2581ec2466e2a02cb6a67bce340b
  Stored in directory: /private/var/folders/jg/w_b1r4z113v36vwhx9d2d8k05zj9c7/T/pip-ephem-wheel-cache-bx0gj4c6/wheels/db/e1/a3/c5cc10ec08b93503cc9641e791cfdc9e53a6426cca12e9fb8b
Failed to build debater-python-api
  Attempting uninstall: debater-python-api
    Found existing installation: debater-python-api 3.5.8.RELEASE
    Uninstalling debater-python-api-3.5.8.RELEASE:
      Successfully uninstalled debater-python-api-3.5.8.RELEASE
    Running setup.py install for debater-python-api ... done
  DEPRECATION: debater-python-api was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


### Initialize the Debater API 

The next segment initializes the Debater API.  It assumes the DEBATER_API_KEY environment variable was set with the Project Debater Early Access Program API key, before the notebook was loaded.

In [5]:
# (C) Copyright IBM Corp. 2020.
import os
from debater_python_api.api.clients.narrative_generation_client import Polarity
from debater_python_api.api.debater_api import DebaterApi
api_key = os.environ['DEBATER_API_KEY']
debater_api = DebaterApi(api_key)

The following are some utility functions to pretty print results.

In [6]:
from IPython.core.display import HTML,display
import math
def print_candidates(title,candidates,fields = None):
    display(HTML('<h2>'+title+'</h2>'))
    print('Number of candidates: {}'.format(len(candidates)))
    html = "<table>"
    html += "<tr>"
    if fields == None:
        fields = candidates[0]
    for field in fields:
       html += "<td><p style=\"text-align:left;\">%s</p></td>"%(field)
    html += "</tr>"

    for row in candidates:
        html += "<tr>"
        for field in fields:
            html += "<td><p style=\"text-align:left;\">%s</p></td>"%(row[field])
        html += "</tr>"
    html += "</table>"
    display(HTML(html))     

def print_top_by_function(title,candidates,function, n = 20, fields = None):
    sorted_candidates = sorted(candidates, key=function)[0:n]
    print_candidates(title,sorted_candidates,fields)
    
    

### Retrieve from Wikipedia

First, we use Index Searcher service to retrieve arguments relevant to topic.  We perform two queries to retrieve sentences.  The first query returns sentences that return sentences that contain the word 'that' followed by the main concept of the topic.  For example, "Classics experts ... concluding ... THAT ... in the ROMAN EMPIRE...". The second query returns sentences that contain the main concept followed by a sentiment or causility word from a prepared lexicon. For example, "The ROMAN EMPIRE had an IMMENSE impact on..."

In [37]:
from debater_python_api.api.sentence_level_index.client.sentence_query_base import SimpleQuery
from debater_python_api.api.sentence_level_index.client.sentence_query_request import SentenceQueryRequest
searcher = debater_api.get_index_searcher_client()
candidates = set()
query_size=10000
query = SimpleQuery(is_ordered=True, window_size=12)
query.add_normalized_element(['that'])
query.add_concept_element([dc])
query_request = SentenceQueryRequest(query=query.get_sentence_query(), size=query_size, sentenceLength=(7, 60))
results = searcher.run(query_request)
print("'that' followed by {} appears {} times. ".format(dc,len(results)))
candidates.update(results)

  
query = SimpleQuery(is_ordered=False, window_size=12)
query.add_concept_element([dc])
query.add_type_element(['Causality', 'Sentiment'])
query_request = SentenceQueryRequest(query=query.get_sentence_query(), size=query_size, sentenceLength=(7, 60))
results = searcher.run(query_request)
print("{} followed by sentiment or causality word appears {} times. ".format(dc,len(results)))
candidates.update(results)

texts = ([ { 'news_content': c, 'news_title' : c, 'id' : c } for c in candidates ])



'that' followed by Roman Empire appears 1589 times. 
Roman Empire followed by sentiment or causality word appears 8472 times. 


Save the results to file.

In [14]:
import json
with open(articles_filename, 'w') as outfile:
    json.dump(texts, outfile)  
 

### Read the articles from the saved file

Next we, read the articles from the saved file in json format, split into to sentences, and create an html snippet which includes the sentence, the surrounding context and the article's title.  We also remove duplicate sentences.

In [15]:
import json
with open(articles_filename, "r") as read_file:
    data = json.load(read_file)

from spacy.lang.en import English # updated
import re
nlp = English()

all_sentences = []
hashed_sentences = {}
total_sentences = 0
nlp.max_length = 1000000000
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
for hit in data:
    doc_id = hit['id']
    doc_text = hit['news_content']
    doc_title = hit['news_title']
    doc = nlp(doc_text)
    prev = {'text' : '', 'html':''}
    for (i,sent) in enumerate(doc.sents):
        total_sentences += 1
        text = sent.string.strip()
        hash_of_text = hash(text)
        if hash_of_text in hashed_sentences:
            continue
        hashed_sentences[hash_of_text] = True    
        sentence_entry = { 'doc_id': doc_id , 'line' : i , 'text': text , 'prev': prev['text'] }
        sentence_entry['wordcount'] = len(re.findall(r'\w+', sentence_entry['text']))
        prev[ 'next' ] = sentence_entry['text']
        sentence_entry[ 'html'] = sentence_entry['prev'] + '<b><br>' + sentence_entry['text'] + '</b><br>' 
        prev[ 'html' ] += sentence_entry['text']
        prev = sentence_entry
        all_sentences.append(sentence_entry)
    sentence_entry['next'] = ''      
print('Number of removed duplicates: {}'.format(total_sentences - len(all_sentences)))
print('Total number of sentences remaining: {}'.format(len(all_sentences)))



Number of removed duplicates: 65
Total number of sentences remaining: 9473


We now filter sentences that contain the main concept using a regular expression. We also remove sentences that are too long.


In [ ]:
import re
all_candidates = [sent for sent in all_sentences  if re.search(dc_re,sent['text'],re.IGNORECASE) and sent['wordcount'] < max_sentence_length]
print('Total number of sentences containing the topic:' + str(len(all_candidates)))


We print some random example candidates.  We can see most are not argumentative.

In [38]:
print_candidates("Example candidates", all_candidates[0:100], ['html'])

Number of candidates: 100


html
"Christianity eventually became the dominant religion of the Roman Empire, and Zeus decided that the Olympian gods would break most of their ties with Earth, except for Poseidon, who would continue to watch over his worshippers in Atlantis."
"Within the ""tale"" sections of the book, Peter, Martin, and Jack fall into bad company (becoming the official religion of the Roman empire) and begin altering their coats (faith) by adding ornaments."
"In the 1951 film Quo Vadis, Nero's repeated use of the salute at mass rallies explicitly presents the Roman Empire as a Fascist military state [REF]."
"Europe has about 225 indigenous languages,[REF] mostly falling within three Indo-European language groups: the Romance languages, derived from the Latin of the Roman Empire; the Germanic languages, whose ancestor language came from southern Scandinavia; and the Slavic languages [REF]."
"zemenyei road to the remains of the highway, built by the Roman Empire from Rome Arrabonáig, that today's Gyor."
"Not long after, Wild Geese Gate is threatened by a legion from the Roman Empire in need of water and supplies, among which is a blind boy named Publius (Jozef Waite)."
"Archaeological finds have revealed that Empoli was already settled in the early Roman Empire times, and continued to exist until the 4th century AD."
This opened the Silk Road and led to the appearance of Chinese silk in the Roman Empire.
"Historically, master morality was defeated, as the slave morality of Judaism and Christianity spread throughout the Roman Empire."
The Ottomans were one of the first states to maintain a standing army in Europe since the Roman Empire.


### Finding most argumentative statements

We now score the candidate sentences' stance toward the topic using the Debater Pro/Con service.  

In [17]:
sentence_topic_dicts = [{'sentence' : sentence['text'], 'topic' : topic } for sentence in all_candidates]
pro_con_scores = debater_api.get_pro_con_client().run(sentence_topic_dicts)
for sentence, pro_con_score in zip(all_candidates, pro_con_scores):
    sentence['procon'] = round(pro_con_score, 3)


ProConClient: 100%|██████████| 4375/4375 [01:23<00:00, 52.12it/s]

Similarly, we score the candidate sentences according to the Debater Evidence Detection service.

In [18]:
evidence_scores = debater_api.get_evidence_detection_client().run(sentence_topic_dicts)
for sentence, evidence_score in zip(all_candidates, evidence_scores):
    sentence['evidence'] = round(evidence_score, 3)


EvidenceDetectionClient: 100%|██████████| 4375/4375 [09:16<00:00,  7.72it/s]

We now print the top 20 statements that have the highest combination of evidence and stance scores.

In [19]:
print_top_by_function("Most argumentative",all_candidates,
                          function=lambda candidate: -abs(candidate['procon'])-candidate['evidence'],
                                                       fields = ['html','procon','evidence'])

Number of candidates: 20


html,procon,evidence
Mussolini wanted to re-establish the greatness of the Roman Empire and believed that Italy was the most powerful of the Mediterranean countries after World War I [REF].,0.973,0.851
"In the first century BC, the Roman Empire emerged as the dominant power in the Mediterranean Basin and became the leading cultural, political and religious centre of Western civilisation.",0.969,0.832
"That, plus the proliferation of Latin throughout the world (filtered through French and English) makes the Roman Empire by far the most successful empire ever.",0.984,0.805
The expansion of the Roman Empire helped to broaden trade patterns and the introduction of the camel as means of transport significantly fostered the spread of imported goods [REF].,0.898,0.887
"Classics experts have examined the status of women in the ancient world, concluding that in the Roman Empire, with its superior social organization, internal peace, and rule of law, allowed women to enjoy a somewhat better standing than in ancient Greece, where women were distinctly inferior [REF].",0.995,0.77
"For both the Pope and Charlemagne, the Roman Empire remained a significant power in European politics at this time.",0.95,0.811
"In 27 BC, Augustus established the Roman Empire and inaugurated the Pax Romana, a period of stability and relative peace in which Italy flourished as the leading political and economic centre of the known world.",0.951,0.764
"Ataulf famously declared that he had abandoned his intention to set up a Gothic empire because of the irredeemable barbarity of his followers, and instead he sought to restore the Roman Empire.",0.89,0.821
"In contrast with the declining empire theories, historians such as Arnold J. Toynbee and James Burke argue that the Roman Empire itself was a rotten system from its inception, and that the entire Imperial era was one of steady decay of institutions founded in Republican times.",-0.984,0.7
The Roman Empire had an immense impact on the development of viticulture and oenology.,0.986,0.669


### Generate narrative

We now use the narrative generation service to automatically generate a short supporting narrative for the topic, based on the most argumentative evidence found.

In [33]:
# customize opening statement
opening_statement_customization_dict = {
    'type' : 'openingStatement',
    'items':
        [
              {
                'key': 'Opening_statement',
                'value': ' The following speech is based on <NUM_SBC_ARGUMENTS_SUGGESTED> mined from Wikipedia supporting the notion that <MOTION>.',
                'itemType' : 'single_string'
              },        
        ]
}

print('Generating speech:')

sentences = [ sentence['text'] for sentence in all_candidates if sentence['evidence'] > 0.2]
print('Number of sentences passed to speech generation: ' + str(len(sentences)))
narrative_generation_client = debater_api.get_narrative_generation_client()
sentence_topic_dicts = [{'sentence' : sentence, 'topic' : topic } for sentence in sentences]
pro_con_scores = debater_api.get_pro_con_client().run(sentence_topic_dicts)
speech = narrative_generation_client.run(topic=topic, dc=dc, sentences=sentences,
                                         pro_con_scores=pro_con_scores, polarity=Polarity.PRO, 
                                         customizations=[opening_statement_customization_dict])
print('\n\nSpeech:' + speech.status)
if (speech.status == 'DONE'):
    print(speech)
else: 
    print(speech.error_message)




ProConClient:   0%|          | 0/178 [00:00<?, ?it/s]

Generating speech:
Number of sentences passed to speech generation: 178




ProConClient: 100%|██████████| 178/178 [00:01<00:00, 103.44it/s]

25/07/2021 16:30:44, SpeechResultResponse=60fd6784ed7ca41fdb88a082, motionGenerationid=0, status=RUNNING, started to wait at 25/07/2021 16:30:43


ProConClient: 100%|██████████| 178/178 [00:14<00:00, 12.40it/s] 



Speech:DONE
The following speech is based on 178 mined from Wikipedia supporting the notion that The Roman Empire was beneficial to the world. 

The crowd raised four issues. The first claim is that the Roman Empire played a vital role. In addition, we will hear about politics, regions and effectiveness. 

Regarding role.  The Romans soon spread east, taking Greece, and the Greek heritage played an important role in the Roman Empire.   For example, the Roman Empire played a vital role in building up barbarian groups along its frontier.   The idea of a Christian empire continued to play a powerful role in Western Europe even after the collapse of Roman rule there; the name of the Holy Roman Empire bears witness to its claims to sanctity as well as to universal rule.  

Moving on to politics.  The Roman Empire was among the most powerful economic, cultural, political and military forces in the world of its time, and it was one of the largest empires in world history.   In the first cen

### Analyze statements per user provided aspects

The next step is to extract for each sentence all the wikipedia concepts mentioned in it using the TermWikifier service.

In [21]:
def add_sentence_mentions(sentences):
    term_wikifier_client = debater_api.get_term_wikifier_client()
    mentions_list = term_wikifier_client.run([sentence['text'] for sentence in sentences])
    for sentence, mentions in zip(sentences, mentions_list):
        sentence['mentions'] = set([mention['concept']['title'] for mention in mentions])
    all_mentions = set([mention for sentence in sentences for mention in sentence['mentions']])
    return set(all_mentions)

all_mentions = add_sentence_mentions(all_candidates)    
print('Total number of unique concepts: {}'.format(len(all_mentions)))

TermWikifierClient: 100%|██████████| 4375/4375 [00:36<00:00, 118.75it/s]

Total number of unique concepts: 5862


For each aspect, we find all closely related wikipedia concepts (that are mentioned in the given sentences).

In [22]:
def get_related_mentions(concept, threshold, mentions):
    term_relater_client = debater_api.get_term_relater_client()
    concept_mention_pairs = [[concept, mention] for mention in mentions]
    scores = term_relater_client.run(concept_mention_pairs)
    return [mention for mention, score in zip(all_mentions, scores) if score > threshold]


In [23]:
matched_mentions = {}
for aspect in aspects:
    matched_mentions[aspect] = get_related_mentions(aspect, 0.8, all_mentions)
    print(aspect,":",matched_mentions[aspect])
    
    

TermRelaterClient:   0%|          | 0/5862 [00:00<?, ?it/s]

War : ['Wargame', 'Civil war', 'Real-time tactics', 'Peace treaty', 'Marcomannic Wars', 'Gallic Wars', 'Military service', 'Front (military)', 'Military history', 'Combat', 'Victory', 'Military administration', 'German Army', 'Military tactics', 'Declaration of war', 'Military strategy', 'Second Punic War', 'First Punic War', 'Combatant', 'Army', 'Invasion', 'Battle', 'Military campaign', 'Private army', 'Religious war', 'Soldier', 'American Civil War', 'Just war theory', 'Military tradition', 'Ancient Greek warfare', "Caesar's Civil War", 'War', 'Artillery', 'Prize of war', 'Prisoner of war', 'Korean War', 'Peace', 'Military history of the United States', 'Violence', 'Illyrian Wars', 'Military organization', 'Military', 'Nuclear weapon', 'Enemy', 'Third Servile War', 'War flag', 'Military threat', 'Strategic goal (military)', "Fitzpatrick's War", 'Peloponnesian War', 'Military technology', 'Battlefield', 'Guerrilla warfare', 'Mercenary', 'Peace process', 'Naval warfare', 'Armed', 'Mil


TermRelaterClient: 100%|██████████| 5862/5862 [00:58<00:00, 99.94it/s]

TermRelaterClient:   0%|          | 0/5862 [00:00<?, ?it/s]

Economy : ['Economic policy', 'Economics', 'Economic system', 'Economic inequality', 'Economic development', 'Economic expansion', 'Economic growth', 'Market (economics)', 'Profit (economics)', 'Economy of Portugal', 'Industry (economics)', 'Capital (economics)', 'Commodity', 'World economy', 'Economy', 'Economic depression', 'Market economy']



TermRelaterClient: 100%|██████████| 5862/5862 [00:58<00:00, 97.17it/s]

Technology : ['Inventor', 'Technology', 'Manufacturing', 'Engineering', 'Scientist', 'Innovation', 'Military technology', 'Science', 'Machine', 'Information technology', 'Computer', 'Invention', 'Massachusetts Institute of Technology', 'Robot']


We identify for each sentence all related concepts that appear in it for each aspect. 

In [24]:
for sentence in all_candidates:
    for aspect in aspects:
        sentence[aspect + '_concepts'] = sentence['mentions'].intersection(matched_mentions[aspect]) 
 

TermRelaterClient: 100%|██████████| 5862/5862 [00:58<00:00, 99.55it/s]



We now generate the final output.  We print the top 20 most relevant sentences for each aspect.

We first focus on sentences that contain atleast one related concept to the aspect. 

We then print the top sentences sorted by an aggregated score based on their evidence score, the pro con score, and the number of related concepts in the sentence.

In [25]:
for aspect in aspects:
    matched_sentences = [sentence for sentence in all_candidates if len(sentence[aspect + '_concepts']) >= 1]
           
    for sentence in all_candidates:
        sentence['score'] = len(sentence[aspect + '_concepts'])/3 + sentence['evidence'] + abs(sentence['procon'])
          
    print_top_by_function(aspect,matched_sentences,
                          function=lambda candidate: -candidate['score'],
                                                       fields = ['html','procon', 'evidence',aspect + '_concepts'])

Number of candidates: 20


html,procon,evidence,War_concepts
"Classics experts have examined the status of women in the ancient world, concluding that in the Roman Empire, with its superior social organization, internal peace, and rule of law, allowed women to enjoy a somewhat better standing than in ancient Greece, where women were distinctly inferior [REF].",0.995,0.77,{'Peace'}
"In 27 BC, Augustus established the Roman Empire and inaugurated the Pax Romana, a period of stability and relative peace in which Italy flourished as the leading political and economic centre of the known world.",0.951,0.764,{'Peace'}
"MacDonald emphasizes negative public opinion in the Roman empire produced violence and impacted the church's ability to ""evangelize"" [REF].",-0.74,0.851,{'Violence'}
"The Roman Empire was among the most powerful economic, cultural, political and military forces in the world of its time, and it was one of the largest empires in world history.",0.994,0.436,{'Military'}
Its regular practice during the Roman Republic and Roman Empire led to military excellence and victory.,0.969,0.119,"{'Victory', 'Military'}"
"The Roman Empire was among the most powerful economic, cultural, political and military forces in the world of its time.",0.994,0.41,{'Military'}
"In 31 BC, the great naval Battle of Actium ended the civil wars culminating in the final victory of Augustus and the establishment of the Roman Empire.",0.766,0.275,"{'Victory', 'Civil war'}"
"The Roman Empire, which then ruled over neighbouring southern Britain, represented a very real threat of invasion to Ireland, which may have acted as a spur to the creation of the alphabet.",-0.809,0.551,{'Invasion'}
"During the Crisis of the Third Century the Roman Empire nearly collapsed under the combined pressures of invasions, military anarchy and civil wars, and hyperinflation.",-0.62,0.049,"{'Civil war', 'Military', 'Invasion'}"
He increases taxes in order to maintain the war against the Sassanids and strengthen the defenses of the Roman Empire.,0.928,0.06,"{'War', 'Military'}"


Number of candidates: 20


html,procon,evidence,Economy_concepts
"In 27 BC, Augustus established the Roman Empire and inaugurated the Pax Romana, a period of stability and relative peace in which Italy flourished as the leading political and economic centre of the known world.",0.951,0.764,{'Economy'}
"The period of Ancient Greece 4th century B.C. and later of the Roman Empire marks the beginning not only of democracy, but as well as its connection to economic growth.",0.979,0.499,{'Economic growth'}
"The Roman Empire was among the most powerful economic, cultural, political and military forces in the world of its time, and it was one of the largest empires in world history.",0.994,0.436,{'Economy'}
"The Roman Empire was among the most powerful economic, cultural, political and military forces in the world of its time.",0.994,0.41,{'Economy'}
"Deforestation during the Roman period was a result of the geographical expansion of the Roman Empire, with its increased population, large-scale agriculture, and unprecedented economic development.",0.825,0.548,{'Economic development'}
"Institutions of the Roman Empire government failed to meet the moral and economic needs of the citizens, resulting in the conditions that would facilitate political decay and the fall of the Roman state [REF].",-0.991,0.367,{'Economy'}
"In the past, geographically large economic systems existed but were mostly limited to spheres of domination of large empires (such as the Roman Empire); development of capitalism enabled the world economy to extend beyond individual states [REF].",0.843,0.138,"{'Economic system', 'World economy'}"
"For example, economics played a key role in the Roman Empire.",0.854,0.252,{'Economics'}
Historian Michael Rostovtzeff and economist Ludwig von Mises both argued that unsound economic policies played a key role in the impoverishment and decay of the Roman Empire.,-0.93,0.154,{'Economic policy'}
"During the classical age, the unification of China and the pacification of the Mediterranean basin by the Roman Empire created vast regional markets in commodities at either end of Eurasia.",0.582,0.097,"{'Commodity', 'Market (economics)'}"


Number of candidates: 20


html,procon,evidence,Technology_concepts
"Ancient Greek science, philosophy, democracy, architecture, literature, and art provided a foundation embraced and built upon by the Roman Empire as it swept up Europe, including the Hellenic World in its conquests in the 1st century BCE.",0.971,0.458,{'Science'}
"Ancient Greek science, philosophy, democracy, architecture, literature, and art provided a foundation embraced and built upon by the Roman Empire as it swept up Europe, including the Hellenic world in its conquests in the 1st century BC.",0.968,0.457,{'Science'}
Winemaking technology improved considerably during the time of the Roman Empire.,0.856,0.509,{'Technology'}
"The invention of glassblowing coincided with the establishment of the Roman Empire in the 1st century BC, which enhanced the spread and dominance of this new technology [REF].",0.885,0.368,{'Technology'}
"The Roman Empire was one of the most technologically advanced civilizations of antiquity, with some of the more advanced concepts and inventions forgotten during the turbulent eras of Late Antiquity and the early Middle Ages.",0.955,0.193,{'Invention'}
"Legacy of the Roman Empire - what the Roman Empire passed on, in the form of cultural values, religious beliefs, as well as technological and other achievements, and through which it continued to shape other civilizations, a process which continues to this day.",0.953,0.174,{'Technology'}
Technological influences spread throughout Europe with the expansion of the Roman empire.,0.86,0.262,{'Technology'}
The Roman Empire had an advanced set of technology for their time.,0.926,0.136,{'Technology'}
"With the rise of the Roman Empire, technology became more popular than philosophy and the next bio-mechanic arose.",0.931,0.112,{'Technology'}
These innovations would be combined and adopted under the Roman empire.,0.887,0.111,{'Innovation'}
